In [6]:
from collections import defaultdict
import numpy as np
import heapq
from collections import defaultdict, Counter
import struct
import sys

ModuleNotFoundError: No module named 'numpy'

# Tirar os residuos/  Pegar os resíduos e transformar na imagem original - Giovana 

# Fazer a tranformada/fazer a transformada inversa - Fernando


# Codificar/Decodidicar (huffman) - Araju


In [ ]:
from heapq import heappush, heappop, heapify
from collections import defaultdict

def encode(symb2freq):
    """Huffman encode the given dict mapping symbols to weights"""
    heap = [[wt, [sym, ""]] for sym, wt in symb2freq.items()]
    heapify(heap)
    while len(heap) > 1:
        lo = heappop(heap)
        hi = heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    return sorted(heappop(heap)[1:], key=lambda p: (len(p[-1]), p))
encode({1 : 1/9, 2 : 1/9, 3 : 1/9, 4 : 1/9, 5 : 1/9, 6 : 1/9, 7 : 1/9, 8 : 1/9, 9 : 1/9})

[[3, '000'],
 [4, '001'],
 [5, '010'],
 [6, '011'],
 [7, '100'],
 [8, '101'],
 [9, '110'],
 [1, '1110'],
 [2, '1111']]

In [ ]:
class Node:
    def __init__(self, freq, symbol=None, left=None, right=None):
        self.freq = freq
        self.symbol = symbol
        self.left = left
        self.right = right

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(frequencies):
    heap = [Node(freq, symbol) for symbol, freq in frequencies.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = Node(left.freq + right.freq, left=left, right=right)
        heapq.heappush(heap, merged)

    return heap[0] if heap else None

def generate_huffman_codes(node, prefix="", codebook=None):
    if codebook is None:
        codebook = {}

    if node is not None:
        if node.symbol is not None:
            codebook[node.symbol] = prefix
        generate_huffman_codes(node.left, prefix + "0", codebook)
        generate_huffman_codes(node.right, prefix + "1", codebook)

    return codebook


{5: '000', 2: '001', 1: '010', 4: '011', 8: '100', 6: '101', 9: '110', 7: '1110', 3: '1111'}


# Salvar no arquivo a imagem comprimida/ descomprimir a imagem -  Araju


In [ ]:
def encode_data(data, huffman_codes):
    return ''.join(huffman_codes[symbol] for symbol in data)

def decode_data(encoded_data, huffman_tree):
    decoded_data = []
    current_node = huffman_tree
    for bit in encoded_data:
        if bit == '0':
            current_node = current_node.left
        else:
            current_node = current_node.right

        if current_node.symbol is not None:
            decoded_data.append(current_node.symbol)
            current_node = huffman_tree

    return decoded_data

def save_to_file(filename, huffman_codes, encoded_data):
    with open(filename, 'wb') as file:
        # Save the number of unique symbols
        file.write(struct.pack('<I', len(huffman_codes)))

        # Save the Huffman codes
        for symbol, code in huffman_codes.items():
            file.write(struct.pack('<i', symbol))  # Symbol as 4 bytes
            file.write(struct.pack('<B', len(code)))  # Length of the code
            file.write(code.encode())  # Code as bytes

        # Save the length of the encoded data in bits
        file.write(struct.pack('<I', len(encoded_data)))

        # Convert the encoded data to bytes
        buffer = 0
        buffer_length = 0
        byte_array = bytearray()

        for bit in encoded_data:
            buffer = (buffer << 1) | int(bit)
            buffer_length += 1

            if buffer_length == 8:
                byte_array.append(buffer)
                buffer = 0
                buffer_length = 0

        # If there are remaining bits in the buffer, pad with zeros and add to byte array
        if buffer_length > 0:
            buffer <<= (8 - buffer_length)
            byte_array.append(buffer)
        print(sys.getsizeof(byte_array))
        file.write(byte_array)

def load_from_file(filename):
    with open(filename, 'rb') as file:
        # Read the number of unique symbols
        num_symbols = struct.unpack('<I', file.read(4))[0]

        # Read the Huffman codes
        huffman_codes = {}
        for _ in range(num_symbols):
            symbol = struct.unpack('<i', file.read(4))[0]
            code_length = struct.unpack('<B', file.read(1))[0]
            code = file.read(code_length).decode()
            huffman_codes[symbol] = code

        # Read the length of the encoded data in bits
        encoded_data_length = struct.unpack('<I', file.read(4))[0]

        # Read the encoded data as bits
        encoded_data = ''
        while len(encoded_data) < encoded_data_length:
            byte = file.read(1)
            if not byte:
                break
            byte_value = ord(byte)
            encoded_data += f'{byte_value:08b}'

        return huffman_codes, encoded_data[:encoded_data_length]

def huffman_encode_matrix(matrix):
    flat_list = [item for sublist in matrix for item in sublist]
    frequencies = Counter(flat_list)
    huffman_tree = build_huffman_tree(frequencies)
    huffman_codes = generate_huffman_codes(huffman_tree)
    encoded_data = encode_data(flat_list, huffman_codes)
    return huffman_codes, encoded_data, huffman_tree

def huffman_decode_matrix(encoded_data, huffman_tree, original_shape):
    decoded_flat_list = decode_data(encoded_data, huffman_tree)
    matrix = []
    index = 0
    for row_size in original_shape:
        matrix.append(decoded_flat_list[index:index + row_size])
        index += row_size
    return matrix

In [ ]:
# Example usage:
matrix = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]
print(sys.getsizeof(matrix))
original_shape = [len(row) for row in matrix]
huffman_codes, encoded_data, huffman_tree = huffman_encode_matrix(matrix)
filename = 'matrix.huff'
save_to_file(filename, huffman_codes, encoded_data)

loaded_huffman_codes, loaded_encoded_data = load_from_file(filename)
decoded_matrix = huffman_decode_matrix(loaded_encoded_data, huffman_tree, original_shape)
print(decoded_matrix)